In [1]:
import pygame
import random
import math
from queue import PriorityQueue
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout
import h5py
import numpy as np
import pandas as pd

# load the neural network from disk
neural_net = load_model('models/big_net_ball_size_30.h5')

# game parameter initialization:

#window size
display_width =  720 
display_height = 620

#table boundaries
top = 10
right = 400
left = 10
bottom = 600

#colors
black = (0,0,0)
white = (255,255,255)
red = (255,0,0)
green = (0,255,0) 
blue = (0,0,255)
dark_green = (30,90,5) 

# to keep score 
score_board = {'score': [0, 0], 'player': ['Player-1', 'Player-2']}  

# game physics 
friction = 0.012
elasticity = 0.85
stop_speed = 0.5
max_speed = 10.0 

# some definitions of angles for initializing data:
degrees = [x for x in range(271)]
radians = [math.radians(a) for a in degrees]
degrees2 = [x for x in range(-89, 0)]
radians2 = [math.radians(a) for a in degrees2]
rads =  radians + radians2
speed_range = [x for x in range(1,11)]

# the ball class 
class Ball:
    
    def __init__(self, size = 30, position = (int(right/2),int(bottom/2)), color = (0,0,0), speed = 0.0, angle = 0):
        self.size =  size
        self.x = position[0]
        self.y = position[1]
        self.color = color
        self.speed = speed 
        self.angle = angle
        self.selected = False
        self.a = None
        self.b = None
        
    def modulo_angle(self, angle_in_rad):
        degrees = math.degrees(angle_in_rad)
        degrees = degrees % 360 
        modulated_angle = math.radians(degrees)
        return modulated_angle

    def display(self, a = None,b = None):
        pygame.draw.circle(game_display,  self.color, (int(self.x), int(self.y)), self.size)
        if self.selected:
            pygame.draw.line(game_display, green, (self.x,self.y), (self.a, self.b), 3)

    def move(self):
        self.x += math.sin(self.angle) * self.speed
        self.y -= math.cos(self.angle) * self.speed
        self.speed -= friction 
        if self.speed < stop_speed:
            self.speed = 0.0 
            #self.angle = 0
        if self.speed > max_speed:
            self.speed = max_speed

    def bounce(self):
        if self.x >= (right + left) - self.size:
            self.x = 2*((right + left) - self.size) - self.x
            self.angle = self.modulo_angle(-self.angle)
            self.speed *= elasticity

        elif self.x <= self.size + left:
            self.x = 2*(self.size + left) - self.x
            self.angle = self.modulo_angle(-self.angle)
            self.speed *= elasticity

        if self.y >= (bottom + top) - self.size:
            self.y = 2*((bottom + top) - self.size) - self.y
            self.angle = self.modulo_angle(math.pi - self.angle)
            self.speed *= elasticity

        elif self.y <= self.size + top:
            self.y = 2*(self.size + top) - self.y
            self.angle = self.modulo_angle(math.pi - self.angle)
            self.speed *= elasticity    

# functions for the game

def text_objects(score, font):
    text_surface = font.render(score, True, black) 
    return text_surface, text_surface.get_rect()
    

def message_display(score_board):
    text = pygame.font.Font('freesansbold.ttf', 20)
    text_surface_1, text_frame_1 = text_objects(score_board['player'][0] +' : '+ str(score_board['score'][0]), text)
    text_surface_2, text_frame_2 = text_objects(score_board['player'][1] +' : '+ str(score_board['score'][1]), text)
    text_frame_1.center = (((display_width - 100), (100)))
    text_frame_2.center = (((display_width - 100), (125)))
    game_display.blit(text_surface_1, text_frame_1)
    game_display.blit(text_surface_2, text_frame_2) 

def angle_display(ball):
    text = pygame.font.Font('freesansbold.ttf', 20)
    text_surface_1, text_frame_1 = text_objects(str(ball.angle), text)
    text_frame_1.center = ((ball.x, ball.y))
    game_display.blit(text_surface_1, text_frame_1)

def random_pos():
    x_pos = random.randint(left+20, right-20) 
    y_pos = random.randint(top+20,bottom-20)
    return (x_pos,y_pos)

def q_ball_select(q_ball, x, y):
    if math.hypot(q_ball.x - x, q_ball.y - y) <= q_ball.size:
        q_ball.selected = True
        q_ball.color = blue 
        q_ball.speed = 0
        q_ball.a = x 
        q_ball.b = y

def q_ball_unselect(q_ball):
    q_ball.selected = False
    q_ball.color = white
    q_ball.a = None 
    q_ball.b = None 

def q_ball_setab(q_ball, x, y):
    q_ball.a = x 
    q_ball.b = y 

# Collision physics and a_star
def coord_distance(pos1,pos2):
    dx = pos1[0] - pos2[0] 
    dy = pos1[1] - pos2[1]
    distance = math.hypot(dx, dy)
    return distance

def collision_check(ball1, ball2):
    distance = coord_distance((ball1.x,ball1.y), (ball2.x,ball2.y))
    return distance <= (ball1.size + ball2.size) 

def get_neighbors(pos):
    x = pos[0]
    y = pos[1] 
    neighbors = [(x+1,y), (x-1,y), (x,y+1), (x,y-1),
                (x+1,y+1), (x+1,y-1), (x-1,y+1), (x-1,y-1)]
    return neighbors

# a_star is used to search for the closest position ball2 could take
# to do away with overlap. 
# returns that position for ball2 
def a_star(b1,b2, dist): 
    origin = (b1.x,b1.y)
    goal_dist = 2*b1.size + 2   # the right distance between balls without overlap (+2 --> to be safe!)
    error = abs(goal_dist - dist) #the current 'wrong' distance between the balls - absolute is taken to handle relative ball positions
    start_pos = (b2.x, b2.y)
    the_q = PriorityQueue(maxsize = 800)
    the_q.put((error, start_pos)) # initiating the queue
    while the_q: 
        neighbors = [] 
        curr_error, curr_pos = the_q.get() # returns a "less wrong" position to minimize error (priority queue)
        if curr_error < 1: 
            return curr_pos 
        neighbors = get_neighbors(curr_pos)
        for pos in neighbors:
            new_dist = coord_distance(origin, pos)
            new_error = abs(goal_dist - new_dist) 
            if new_error < 1:
                return pos 
            the_q.put((new_error,pos)) #add neighbor to the queue for further search from them

def modulo_angle(angle_in_rad):
    degrees = math.degrees(angle_in_rad)
    degrees = degrees % 360 
    modulated_angle = math.radians(degrees)
    return modulated_angle

def collide(b1,b2):
    dx = b1.x - b2.x 
    dy = b1.y - b2.y 

    tangent = math.atan2(dy, dx)
    b1.angle = modulo_angle(2*tangent - b1.angle)
    b2.angle = modulo_angle(2*tangent - b2.angle)

    # adjust overlap using a_star to find the best point without overlap
    distance = coord_distance((b1.x,b1.y),(b2.x,b2.y)) 
    new_x_for_b2, new_y_for_b2 = a_star(b1,b2, distance)
    b2.x = new_x_for_b2
    b2.y = new_y_for_b2

    if not collision_check(b1,b2):
        new_speed = elasticity * (b1.speed + b2.speed)/2
        b1.speed = b2.speed = new_speed
        
#function to generate the test data 
# combines a state with all possible actions for neural network to predict the q_value
def generate_test(state, rads, speed_range):
    test_data = []
    for angle in rads:
        for speed in speed_range:
            test_data.append(state+[angle]+[speed])
    test_data = np.array(test_data)
    return test_data

def action_select(test, preds):
    d = {'state_action': test, 'predictions': preds}
    df = pd.DataFrame(d)
    df = df.sort_values(by=['predictions'], ascending = False)
    return df


def game_loop(no_of_rounds=1):

    q_ball = Ball(color = white)
    red_ball = Ball(color = red, position = random_pos(), speed = 5.0)
    black_ball = Ball(color = black, position = random_pos(), speed = 5.0)

    balls = [q_ball, red_ball, black_ball] 

    stop = False
    rounds = 0
    
    player_score = 0
    collided_with_red = False
    collided_with_black = False
    game_round_start = False
    reward = 0

    while not stop:

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                stop = True

            if event.type == pygame.MOUSEBUTTONDOWN:
                mouseX, mouseY = pygame.mouse.get_pos()
                q_ball_select(q_ball, mouseX, mouseY)
                
            if event.type == pygame.MOUSEMOTION and q_ball.selected: 
                mouseX, mouseY = pygame.mouse.get_pos()
                q_ball_setab(q_ball, mouseX, mouseY) 
                

            if event.type == pygame.MOUSEBUTTONUP and q_ball.selected:
                mouseX, mouseY = pygame.mouse.get_pos()
                dx = mouseX - q_ball.x    
                dy = mouseY - q_ball.y  
                q_ball.angle = math.atan2(dy, dx) + 0.5*math.pi
                q_ball.speed = math.hypot(dx, dy) * 0.06
                q_ball_unselect(q_ball)
                

        game_display.fill(white)
        pygame.draw.rect(game_display, dark_green, [left, top, right, bottom])

        if q_ball.speed == 0 and red_ball.speed == 0.0 and black_ball.speed == 0.0:
            collided_with_red = False
            collided_with_black = False 
            
            new_state = [q_ball.x, q_ball.y, red_ball.x, red_ball.y, black_ball.x, black_ball.y]
            test_data = generate_test(new_state, rads, speed_range)
            
            predictions = neural_net.predict(test_data)
            
            q_vals = np.concatenate((test_data, predictions), axis=1)
            q_vals = q_vals[q_vals[:,8].argsort()[::-1]]
            action = q_vals[0] 
            q_ball.angle = action[6]
            q_ball.speed = action[7]
            
            game_round_start = True
            rounds+=1
            reward = 0
            if rounds >= no_of_rounds:
                stop = True

        if collision_check(q_ball, red_ball) and game_round_start:
            collide(q_ball, red_ball)
            collided_with_red = True
            if collided_with_black:
                player_score += 1000
                reward = 100
            else:
                player_score += 10
                reward = 10
                       
        if collision_check(q_ball, black_ball) and game_round_start:
            collide(q_ball, black_ball) 
            collided_with_black = True
            if collided_with_red:
                player_score += 1000
                reward = 100
            else:
                player_score += 10
                reward = 10 

        if collision_check(red_ball, black_ball) and game_round_start:
            collide(red_ball, black_ball)

        score_board['score'][0] = player_score 
        
        for ball in balls:
            ball.move()
            ball.bounce()

        for ball in balls:
            ball.display()
            #angle_display(ball)
            
        if q_ball.speed == 0.0 and red_ball.speed == 0.0 and black_ball.speed == 0.0 and game_round_start:
            #targets.append([reward])
            game_round_start = False

        message_display(score_board)
        pygame.display.update()
        clock.tick(120)


# game initiations:  
pygame.init()
game_display = pygame.display.set_mode((display_width, display_height))
clock = pygame.time.Clock()

game_loop(200)
pygame.quit()



c:\users\mrinal\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


InternalError: Blas GEMM launch failed : a.shape=(32, 8), b.shape=(8, 20), m=32, n=20, k=8
	 [[Node: dense_84/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_dense_84_input_0_0/_497, dense_84/kernel/read)]]
	 [[Node: dense_94/Relu/_499 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_104_dense_94/Relu", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'dense_84/MatMul', defined at:
  File "c:\users\mrinal\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\mrinal\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\mrinal\appdata\local\programs\python\python36\lib\site-pac